In [1]:
# Import dependencies
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

In [15]:
data_df=pd.read_csv('Recruitment_pipeline_coded_cleaner.csv')

C:\Users\sstewart4\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4,19,20,21,22,23,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
data_df.dtypes 

study_id                   object
sex                       float64
birthyear                 float64
initial_date               object
initial_contact_method     object
fu_dt_1                    object
fu_method_1                object
fu_outcome_1               object
fu_dt_2                    object
fu_method_2                object
fu_outcome_2               object
fu_dt_3                    object
fu_method_3                object
fu_outcome_3               object
fu_dt_4                    object
fu_method_4                object
fu_outcome_4               object
fu_dt_5                    object
fu_method_5                object
fu_outcome_5               object
visit_planned_date         object
visit_reminder_date        object
actual_visit_date          object
visit_weekday              object
coded_location            float64
visit_outcome              object
Unnamed: 26                object
Unnamed: 27                object
Unnamed: 28               float64
Unnamed: 29   

In [17]:
### calculating the number of times that the participant was schedualed to show up
data_df['freq']=data_df.groupby('study_id')['study_id'].transform('count') - 1

### condesing all rows that are duplicates on the study_id into one. WARNING this is not perfect and we are losing some data in the visit_planned_date through the visit_outcome data. Right now it just picks either the first or last data that came up in those columns when combinding them
condensed_data=data_df.groupby(data_df['study_id']).aggregate({'study_id': 'first', 'sex': 'first', 'birthyear': 'first', 'initial_date': 'first', 'initial_contact_method': 'first', 'fu_dt_1':  'first', 'fu_method_1': 'first', 'fu_outcome_1': 'first', 'fu_dt_2': 'first'
,'fu_method_2': 'first', 'fu_outcome_2': 'first', 'fu_dt_3':'first', 'fu_method_3': 'first', 'fu_outcome_3': 'first', 'fu_dt_4':'first', 'fu_method_4': 'first', 'fu_outcome_4': 'first',  'fu_dt_5':'first', 'fu_method_5': 'first', 'fu_outcome_5': 'first', 'visit_planned_date': 'last'
, 'visit_reminder_date': 'last', 'actual_visit_date':'last', 'visit_weekday':'last', 'coded_location':'last', 'visit_outcome':'last', 'freq':'first'})

### droping data where visit outcome is na
condensed_data=condensed_data[condensed_data['visit_outcome'].notna()]

### making a data frame of just the contact attempts before the appointment
count_stuff=condensed_data[['fu_dt_1','fu_dt_2', 'fu_dt_3', 'fu_dt_4', 'fu_dt_5']]

### calculating the number of attempts made to contact the participant
condensed_data['contact_count']= 5- count_stuff.isnull().sum(axis=1)

### converting the birth year to a date
condensed_data['birthyear']=pd.to_datetime(condensed_data['birthyear'], format='%Y', exact=False)

### converting the actual_visit_date to a date
condensed_data['actual_visit_date']=pd.to_datetime(condensed_data['actual_visit_date'], format='%m/%d/%Y', exact=False)
condensed_data['initial_date']=pd.to_datetime(condensed_data['initial_date'], format='%m/%d/%Y', exact=False)

### Calculating the age of the person at the time of their visit.... its stuck in days and not years right now :/
#condensed_data['Age_at_visit']=(condensed_data['actual_visit_date']- condensed_data['birthyear'])

condensed_data['birthyear'] = condensed_data.birthyear.dt.year


#condensed_data.age = condensed_data.age.fillna(-1)
condensed_data['time_of_visit_from_planned']= condensed_data['actual_visit_date'] - condensed_data['initial_date']

# condensed_data['new_age'] = condensed_data['age'].apply(np.int64)

condensed_data['year'] = condensed_data.actual_visit_date.dt.year
condensed_data['age']= condensed_data['year'] - condensed_data['birthyear']
# condensed_data['time_fu_dt_2'] = condensed_data['fu_dt_2'].time()

#condensed_data['new_age'] = condensed_data['age'].apply(np.int64)

#condensed_data["date_time_fu_dt_2"] = pd.to_datetime(condensed_data["fu_dt_2"])

#for d in condensed_data['date_time_fu_dt_2']:
 # condensed_data['date_fu_dt_2'] = d.date()
  #condensed_data['time_fu_dt_2'] = d.time()

#condensed_data['date_fu_dt_2'] = pd.to_datetime(condensed_data['date_time_fu_dt_2']).dt.date
#condensed_data['time_fu_dt_2'] = pd.to_datetime(condensed_data['date_time_fu_dt_2']).dt.time

condensed_data['age'] = condensed_data['age'].astype('Int16')
condensed_data


,study_id,sex,birthyear,initial_date,initial_contact_method,fu_dt_1,fu_method_1,fu_outcome_1,fu_dt_2,fu_method_2,...,visit_reminder_date,actual_visit_date,visit_weekday,coded_location,visit_outcome,freq,contact_count,time_of_visit_from_planned,year,age
study_id,,,,,,,,,,,,,,,,,,,,,
0001b470ffdea9daf4e3e25a9cfb93d0,0001b470ffdea9daf4e3e25a9cfb93d0,2.0,1980.0,2021-02-28,passive,3/9/2021 15:26,active,None,4/8/2022 14:38,active,...,4/20/2022,2022-04-21,R,1.0,no_show_cancel,2,5,417 days,2022,42
0003a8af34dd38929b6d0dff83a281e5,0003a8af34dd38929b6d0dff83a281e5,NaN,NaN,2019-10-16,passive,10/16/2019 15:11,active,None,None,None,...,10/21/2019,2019-10-22,T,2.0,complete,1,1,6 days,2019,<NA>
00042cd8cfa979e5cbf9aa79a01ec23d,00042cd8cfa979e5cbf9aa79a01ec23d,2.0,1947.0,2021-06-30,passive,7/8/2021 12:40,active,crc_followup,4/8/2022 12:25,active,...,4/27/2022,2022-04-28,R,1.0,no_show_cancel,3,4,302 days,2022,75
0014065de00658be8463cc209aae5bc1,0014065de00658be8463cc209aae5bc1,2.0,1961.0,2020-01-17,passive,1/21/2020 17:50,active,None,2/24/2022 17:13,active,...,3/21/2022,2022-03-23,W,7.0,no_show_cancel,3,5,796 days,2022,61
001e2c5b018d498cd8498e2b9d0ce0c7,001e2c5b018d498cd8498e2b9d0ce0c7,2.0,1958.0,2020-03-13,passive,None,None,None,None,None,...,7/29/2022,2022-12-05,M,2.0,reschedule,2,0,997 days,2022,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffec65b7a5fd2d36cd56e0b249322d1f,ffec65b7a5fd2d36cd56e0b249322d1f,NaN,1970.0,2020-01-13,active,1/13/2020 9:54,active,None,None,None,...,1/28/2020,2020-01-29,W,2.0,complete,1,1,16 days,2020,50
ffeeb12c35a69dcdd34c4b2db6951789,ffeeb12c35a69dcdd34c4b2db6951789,2.0,1955.0,2019-09-22,passive,10/21/2019 15:31,passive,None,10/23/2019 13:34,active,...,11/14/2019,2019-11-15,F,5.0,reschedule,1,3,54 days,2019,64
ffef6be3cbb0c80342dba653133169df,ffef6be3cbb0c80342dba653133169df,1.0,1933.0,2019-09-28,passive,None,None,None,None,None,...,10/7/2019,2019-10-08,T,7.0,no_show_cancel,1,0,10 days,2019,86


In [18]:
condensed_data.dtypes 

study_id                               object
sex                                   float64
birthyear                             float64
initial_date                   datetime64[ns]
initial_contact_method                 object
fu_dt_1                                object
fu_method_1                            object
fu_outcome_1                           object
fu_dt_2                                object
fu_method_2                            object
fu_outcome_2                           object
fu_dt_3                                object
fu_method_3                            object
fu_outcome_3                           object
fu_dt_4                                object
fu_method_4                            object
fu_outcome_4                           object
fu_dt_5                                object
fu_method_5                            object
fu_outcome_5                           object
visit_planned_date                     object
visit_reminder_date               

In [20]:
count_stuff=condensed_data[['fu_dt_1','fu_dt_2', 'fu_dt_3', 'fu_dt_4', 'fu_dt_5']]

count_stuff=count_stuff.fillna('', axis=1)
#something=count_stuff.T
#column_list=something.columns.tolist()
#something=something.reset_index()
#column_list
#hope=pd.DataFrame()
#something.fillna("",inplace=True)
#for item in column_list:
    #test=something[item].apply(' '.join).reset_index()
new4=count_stuff['fu_dt_4']
new3=count_stuff['fu_dt_3']
new2=count_stuff['fu_dt_2']
new1=count_stuff['fu_dt_1']



count_stuff["combined_date"]= count_stuff['fu_dt_5']
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new4, sep =", ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new3, sep =", ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new2, sep =", ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new1, sep =", ")
condensed_data['combined_date']=count_stuff['combined_date']
condensed_data

contact4=condensed_data['fu_method_4']
contact3=condensed_data['fu_method_3']
contact2=condensed_data['fu_method_2']
contact1=condensed_data['fu_method_1']
condensed_data["contact"]=condensed_data['fu_method_5']
condensed_data["contact"].fillna('',inplace=True)
condensed_data["contact"]=condensed_data["contact"].str.cat(contact4, sep =" ")
condensed_data["contact"].fillna('',inplace=True)
condensed_data["contact"]=condensed_data["contact"].str.cat(contact3, sep =" ")
condensed_data["contact"].fillna('',inplace=True)
condensed_data["contact"]=condensed_data["contact"].str.cat(contact2, sep =" ")
condensed_data["contact"].fillna('',inplace=True)
condensed_data["contact"]=condensed_data["contact"].str.cat(contact1, sep =" ")
condensed_data["contact"].fillna('None',inplace=True)
condensed_data['contact']=condensed_data['contact']
condensed_data['contact']=condensed_data['contact'].str.extract('([^\s]+)')
condensed_data['last_contact_time']=condensed_data['combined_date'].str.extract('(..[:]..)')
#condensed_data['last_contact_time']=condensed_data['combined_date'].str.extract('([\d:]+$)')
condensed_data['last_contact_time']=condensed_data['last_contact_time'].fillna('0:00')
#count_stuff
condensed_data

,study_id,sex,birthyear,initial_date,initial_contact_method,fu_dt_1,fu_method_1,fu_outcome_1,fu_dt_2,fu_method_2,...,coded_location,visit_outcome,freq,contact_count,time_of_visit_from_planned,year,age,combined_date,contact,last_contact_time
study_id,,,,,,,,,,,,,,,,,,,,,
0001b470ffdea9daf4e3e25a9cfb93d0,0001b470ffdea9daf4e3e25a9cfb93d0,2.0,1980.0,2021-02-28,passive,3/9/2021 15:26,active,None,4/8/2022 14:38,active,...,1.0,no_show_cancel,2,5,417 days,2022,42,"5/17/2022 14:56, 4/26/2022 15:46, 4/25/2022 12...",active,14:56
0003a8af34dd38929b6d0dff83a281e5,0003a8af34dd38929b6d0dff83a281e5,NaN,NaN,2019-10-16,passive,10/16/2019 15:11,active,None,None,None,...,2.0,complete,1,1,6 days,2019,<NA>,", , , , 10/16/2019 15:11",active,15:11
00042cd8cfa979e5cbf9aa79a01ec23d,00042cd8cfa979e5cbf9aa79a01ec23d,2.0,1947.0,2021-06-30,passive,7/8/2021 12:40,active,crc_followup,4/8/2022 12:25,active,...,1.0,no_show_cancel,3,4,302 days,2022,75,", 5/5/2022 16:17, 4/13/2022 11:40, 4/8/2022 12...",passive,16:17
0014065de00658be8463cc209aae5bc1,0014065de00658be8463cc209aae5bc1,2.0,1961.0,2020-01-17,passive,1/21/2020 17:50,active,None,2/24/2022 17:13,active,...,7.0,no_show_cancel,3,5,796 days,2022,61,"3/22/2022 16:54, 3/16/2022 13:26, 2/25/2022 11...",passive,16:54
001e2c5b018d498cd8498e2b9d0ce0c7,001e2c5b018d498cd8498e2b9d0ce0c7,2.0,1958.0,2020-03-13,passive,None,None,None,None,None,...,2.0,reschedule,2,0,997 days,2022,64,", , , ,",None,0:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffec65b7a5fd2d36cd56e0b249322d1f,ffec65b7a5fd2d36cd56e0b249322d1f,NaN,1970.0,2020-01-13,active,1/13/2020 9:54,active,None,None,None,...,2.0,complete,1,1,16 days,2020,50,", , , , 1/13/2020 9:54",active,9:54
ffeeb12c35a69dcdd34c4b2db6951789,ffeeb12c35a69dcdd34c4b2db6951789,2.0,1955.0,2019-09-22,passive,10/21/2019 15:31,passive,None,10/23/2019 13:34,active,...,5.0,reschedule,1,3,54 days,2019,64,", , 12/8/2021 17:44, 10/23/2019 13:34, 10/21/2...",passive,17:44
ffef6be3cbb0c80342dba653133169df,ffef6be3cbb0c80342dba653133169df,1.0,1933.0,2019-09-28,passive,None,None,None,None,None,...,7.0,no_show_cancel,1,0,10 days,2019,86,", , , ,",None,0:00


In [21]:
condensed_data['last_contact_time'] = pd.to_datetime(condensed_data['last_contact_time'], format= '%H:%M', exact=False).dt.time
condensed_data_droped=condensed_data.drop(['study_id', 'birthyear', 'initial_date', 'visit_planned_date', 'visit_reminder_date', 'actual_visit_date','fu_dt_1', 'fu_method_1','fu_outcome_1','fu_dt_2', 'fu_method_2','fu_outcome_2','fu_dt_3', 'fu_method_3','fu_outcome_3','fu_dt_4', 'fu_method_4','fu_outcome_4','fu_dt_5', 'fu_method_5','fu_outcome_5','combined_date', 'time_of_visit_from_planned', 'year', 'age', 'last_contact_time'], axis=1)
#condensed_data=condensed_data.drop('study_id', axis=0)
condensed_data_droped

,sex,initial_contact_method,visit_weekday,coded_location,visit_outcome,freq,contact_count,contact
study_id,,,,,,,,
0001b470ffdea9daf4e3e25a9cfb93d0,2.0,passive,R,1.0,no_show_cancel,2,5,active
0003a8af34dd38929b6d0dff83a281e5,NaN,passive,T,2.0,complete,1,1,active
00042cd8cfa979e5cbf9aa79a01ec23d,2.0,passive,R,1.0,no_show_cancel,3,4,passive
0014065de00658be8463cc209aae5bc1,2.0,passive,W,7.0,no_show_cancel,3,5,passive
001e2c5b018d498cd8498e2b9d0ce0c7,2.0,passive,M,2.0,reschedule,2,0,None
...,...,...,...,...,...,...,...,...
ffec65b7a5fd2d36cd56e0b249322d1f,NaN,active,W,2.0,complete,1,1,active
ffeeb12c35a69dcdd34c4b2db6951789,2.0,passive,F,5.0,reschedule,1,3,passive
ffef6be3cbb0c80342dba653133169df,1.0,passive,T,7.0,no_show_cancel,1,0,None


In [26]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Generate our categorical variable lists
application_cat = condensed_data_droped.dtypes[condensed_data_droped.dtypes == "object"].index.tolist()
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(condensed_data_droped[application_cat])) 

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df

c:\Users\sstewart4\Miniconda3\envs\myenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,initial_contact_method_active,initial_contact_method_passive,initial_contact_method_None,visit_weekday_F,visit_weekday_M,visit_weekday_R,visit_weekday_T,visit_weekday_W,visit_weekday_None,visit_outcome_complete,visit_outcome_no_show_cancel,visit_outcome_reschedule,contact_None,contact_active,contact_passive
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9145,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9146,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9147,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
9148,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


### Does the dataframe encode_df need to be used in the model now? 

In [30]:
### Machine learning model we will be using Data currently wont quite fit into this but we are working on that just need to encode the data
# Create our features and target
X = condensed_data.drop(columns="visit_outcome")

y = condensed_data["visit_outcome"]

In [31]:
### spliting our training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(6862, 33)

In [ ]:
### resapling pur data and making our predictions
rf_model = RandomForestClassifier(n_estimators=128, random_state=1) 
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [ ]:
### our balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, predictions)

In [ ]:
### Our confustion matrix
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, predictions)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [117]:
condensed_data.dtypes

study_id                          object
sex                              float64
birthyear                        float64
initial_date                      object
initial_contact_method            object
fu_dt_1                           object
fu_method_1                       object
fu_outcome_1                      object
fu_dt_2                           object
fu_method_2                       object
fu_outcome_2                      object
fu_dt_3                           object
fu_method_3                       object
fu_outcome_3                      object
fu_dt_4                           object
fu_method_4                       object
fu_outcome_4                      object
fu_dt_5                           object
fu_method_5                       object
fu_outcome_5                      object
visit_planned_date                object
visit_reminder_date               object
actual_visit_date         datetime64[ns]
visit_weekday                     object
coded_location  

In [64]:
condensed_data.to_csv('test1.csv')